# The Final Model

Now that we've tried all kinds of different pre-trained models and tuning methods, it's finally time to assemble our very own model based on what we've learned. This model will then serve as the foundation for our application that will be deployed using streamlit.

*Note: To get the exact same model, please follow the steps in the previous notebooks, and pay special attention to the merged and deleted classes. If you're just looking for the model file, it's located at `models/
model_filtered.h5`.*

If you want to use the app right aways, you can do so with `streamlit run main.py`. This will use the model provided with this repo.

In [4]:
!pip install numpy tensorflow matplotlib seaborn scikit-learn 

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input


import sys
sys.path.append('../functions')
from preprocessing import img_dataset_from_dir_and_split_train_val 
from modeling import build_model, unfreeze_model_and_clone
from plot_functions import plot_training_metrics

RSEED = 42
DATASET_PATH = '../data/images/' # Path to the parent folder where the original data is stored
TRAINING_IMAGES = '../data/train' # Path to training images
TESTING_IMAGES = '../data/test' # Path to testing images
CLASSES = [
    'alternaria_leaf_spot',
    'bacterial_blight',
    'bacterial_spot',
    'bacterial_wilt',
    'black_measles',
    'black_rot',
    'blast',
    'brown_spot',
    'brown_streak_disease',
    'citrus_greening',
    'common_rust',
    'early_blight',
    'gray_leaf_spot',
    'healthy',
    'isariopsis_leaf_spot',
    'late_blight',
    'leaf_curl',
    'leaf_mold',
    'mosaic_disease',
    'northern_leaf_blight',
    'powdery_mildew',
    'red_rot',
    'spider_mites',
    'target_spot',
    'tungro',
    ]

In [6]:
# 1. Before working with Keras, it's good practice to clear the session
K.clear_session()

In [7]:
# 2. Next, the train-validation-split takes place. The train-test-split has already been done at the end of notebook 1
train_ds, val_ds = img_dataset_from_dir_and_split_train_val(TRAINING_IMAGES)

NotFoundError: Could not find directory ../data/train

In [ ]:
# 3. We define the model and exclude the top layer

model = keras.applications.EfficientNetB0(
    include_top=False
)

# Optional: Take a look at the model's architecture
# mode.summary()

In [11]:
# 4. Here we build the model with 25 classes
model = build_model(num_classes=25)

In [12]:
# 5. Unfreeze the top 10 layers
unfrozen_model = unfreeze_model_and_clone(model)

epochs = 10
hist = unfrozen_model.fit(train_ds, epochs=epochs, validation_data=val_ds)

In [95]:
# 6. Let's have a look at our metrics
plot_training_metrics(hist)

# ... and heatmap

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix using seaborn heatmap with green color palette
plt.figure(figsize=(14, 14))
sns.heatmap(cm, annot=False, cmap="Greens", xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [97]:
# 7. We then have a look at the classification report

def load_test(data_path):
    ''' 
    Function needs filepath as parameter, it will create a validation dataset of 20% of the total df, 
    Needs an RSEED as global variable,
    Image will be cropped to 1:1 and altered to 224 x 224
    '''
    image_dataset = tf.keras.utils.image_dataset_from_directory(
        data_path,
        image_size = (224, 224),
        crop_to_aspect_ratio = True,
        label_mode = 'categorical',
        shuffle = False
    )
    return image_dataset 

dataset_test_path = '../data/filtered_diseases/test_filtered/'

test_ds = load_test(dataset_test_path)

In [ ]:
# Step 1: Get the true labels from the test dataset
y_true = []
for filepath in test_ds.file_paths:
    label = os.path.basename(os.path.dirname(filepath))
    y_true.append(label)

# Extract unique class labels from your training data
classes = sorted(set(y_true))

# Step 2: Convert true labels to indices using the same mapping used during training
class_to_index = {cls: i for i, cls in enumerate(classes)}
y_true_indices = np.array([class_to_index[label] for label in y_true])

# Step 3: Use your model to make predictions on the test dataset
y_pred_probabilities = patho_model_10.predict(test_ds)

# Step 4: Convert the predicted class probabilities to class labels
y_pred_indices = np.argmax(y_pred_probabilities, axis=1)
y_pred = [classes[i] for i in y_pred_indices]

# Step 5: Generate the classification report
report = classification_report(y_true, y_pred)
print("Classification Report:")
print(report)


In [113]:
# 8. We can have a look at the different class probabilities

def display_class_probabilities(model, img_path, class_names):
    # Load and preprocess the input data
    img = image.load_img(img_path, target_size=(224, 224)) 
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get class probabilities
    probabilities = model.predict(img_array)[0]

    # Display class probabilities with class names
    for class_name, prob in zip(class_names, probabilities):
        print(f"{class_name}: {prob}")


In [ ]:
display_class_probabilities(model_filtered.h5, image_path, CLASSES)

In [ ]:
# 9. Once we're happy with the model, we save it
unfrozen_model.save('patho_model_10.h5')